In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg

from PIL import Image 
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings("ignore")

In [2]:
# data_dir = '../input'
train_dir = 'E:\\kaggle\\sigma data\\asgn\\test\\train\\'
test_dir = 'E:\\kaggle\\sigma data\\asgn\\test\\test\\'

In [3]:
# cv2.imread(train_dir+'img_0.jpg')

In [4]:
# from PIL import Image
# import os, sys

# path = 'E:\\kaggle\\sigma data\\asgn\\test\\test\\'
# dirs = os.listdir( path )

# def resize():
#     for item in dirs:
#         if os.path.isfile(path+item):
#             im = Image.open(path+item)
#             f, e = os.path.splitext(path+item)
#             imResize = im.resize((256,256), Image.ANTIALIAS)
#             imResize.save(f + '.jpg', 'JPEG', quality=90)

# resize()

In [5]:
# from PIL import Image
# import os, sys

# path = 'E:\\kaggle\\sigma data\\asgn\\test\\train\\'
# dirs = os.listdir( path )

# def resize():
#     for item in dirs:
#         if os.path.isfile(path+item):
#             im = Image.open(path+item)
#             f, e = os.path.splitext(path+item)
#             imResize = im.resize((256,256), Image.ANTIALIAS)
#             imResize.save(f + '.jpg', 'JPEG', quality=90)

# resize()

In [6]:
labels = pd.read_csv("train.csv")
# labels.drop([449], inplace = True)
# labels.to_csv('train.csv', index = False)

In [7]:
labels.head()

,name,id
0,img_748.jpg,0
1,img_745.jpg,0
2,img_617.jpg,0
3,img_200.jpg,1
4,img_122.jpg,1


In [8]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = self.df.name[index]
        label = self.df.id[index]
        
        img_path = os.path.join(self.data_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        return image, label

In [9]:
epochs = 30
batch_size = 20
device = torch.device('cuda')
print(device)

cuda


In [10]:
data_transf = transforms.Compose([transforms.ToTensor()])
train_data = ImageData(df = labels, data_dir = train_dir, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size = batch_size)

In [11]:
class Network(nn.Module): 
    def __init__(self):
        # ancestor constructor call
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(4)
        self.fc = nn.Linear(512 * 2 * 2, 2) # !!!
   
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
#         print(x.shape)
        x = x.view(-1, 512 * 2 * 2) # !!!
        x = self.fc(x)
        return x

In [12]:
net = Network().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [13]:
%%time
# Train model
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward
        outputs = net(images)
        loss = loss_func(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 20 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))

Epoch [1/30], Loss: 0.5119
Epoch [1/30], Loss: 0.6638
Epoch [2/30], Loss: 0.5239
Epoch [2/30], Loss: 0.5662
Epoch [3/30], Loss: 0.4174
Epoch [3/30], Loss: 0.5148
Epoch [4/30], Loss: 0.3500
Epoch [4/30], Loss: 0.4724
Epoch [5/30], Loss: 0.2899
Epoch [5/30], Loss: 0.4106
Epoch [6/30], Loss: 0.2084
Epoch [6/30], Loss: 0.3672
Epoch [7/30], Loss: 0.2393
Epoch [7/30], Loss: 0.2618
Epoch [8/30], Loss: 0.3040
Epoch [8/30], Loss: 0.3682
Epoch [9/30], Loss: 0.3041
Epoch [9/30], Loss: 0.3223
Epoch [10/30], Loss: 0.2625
Epoch [10/30], Loss: 0.2621
Epoch [11/30], Loss: 0.2218
Epoch [11/30], Loss: 0.2244
Epoch [12/30], Loss: 0.1603
Epoch [12/30], Loss: 0.1176
Epoch [13/30], Loss: 0.1623
Epoch [13/30], Loss: 0.1477
Epoch [14/30], Loss: 0.2019
Epoch [14/30], Loss: 0.1609
Epoch [15/30], Loss: 0.0942
Epoch [15/30], Loss: 0.0690
Epoch [16/30], Loss: 0.1245
Epoch [16/30], Loss: 0.0753
Epoch [17/30], Loss: 0.0740
Epoch [17/30], Loss: 0.1073
Epoch [18/30], Loss: 0.0693
Epoch [18/30], Loss: 0.0552
Epoch [19/

In [14]:
submit = pd.read_csv('test.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [15]:
predict = []
for batch_i, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    output = net(data)
    
    _, pred = torch.max(output.data, 1)
    predict.append(int(pred))
    
submit['pred'] = predict
submit.to_csv('submission.csv', index=False)

In [16]:
submit.head(40)

,name,id,pred
0,img_130.jpg,1,0
1,img_654.jpg,0,0
2,img_182.jpg,1,1
3,img_691.jpg,0,0
4,img_216.jpg,1,0
5,img_697.jpg,0,1
6,img_527.jpg,1,1
7,img_484.jpg,1,0
8,img_800.jpg,0,0
9,img_1008.jpg,0,1
